In [1]:
import numpy as np
import networkx as nx

In [91]:
# number of simulation nodes
n_high = 5
n_low = 5

# model gdp values
gdp_high = 10
gdp_low = 1

# init model network
def init_model_network(n_high, n_low, gdp_high, gdp_low):
    G = nx.DiGraph()

    G.add_nodes_from(list(range(n_high)), group="H", gdp=gdp_high)
    G.add_nodes_from(list(range(n_high, n_high+n_low)), group="L", gdp=gdp_low)
    
    return G


In [92]:
G = init_model_network(n_high, n_low, gdp_high, gdp_low)

In [198]:
def get_net_payoff(G, origin, a, b, c, d):
    gdp_dict = dict(G.nodes(data="gdp"))
    origin_gdp = gdp_dict.pop(origin)
    
    gdp_diff = [(node, a * (val - origin_gdp)) for node, val in gdp_dict.items()]
    gdp_dest = [b * val for _, val in gdp_dict.items()]
    
    degree_dict = dict(G.in_degree)
    del degree_dict[origin]
    degree_payoffs = [c * val for _, val in degree_dict.items()]
    
    total_payoffs = list(zip(gdp_diff, gdp_dest, degree_payoffs))
    total_payoffs = [(n, v1+v2+v3) for (n, v1), v2, v3 in total_payoffs]
    
    net_payoffs = [(n, v - (d/origin_gdp)) for n,v in total_payoffs]
    return net_payoffs

def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))


from scipy.special import expit

def decide_to_link(net_payoffs, e):
    avg_payoff = np.mean([v for _,v in net_payoffs])
    prob = e * expit(avg_payoff)
    decision = np.random.choice([False,True], size=1, p = [1-prob, prob])[0]
    
    return decision

def pick_dest_node(net_payoffs):
    probs = softmax([v for _,v in net_payoffs])
    dest_node = np.random.choice([n for n,_ in net_payoffs], size=1, p=probs)[0]
    
    return dest_node

def add_edge(G, origin, dest):
    if G.has_edge(origin, dest):
        prev_weight = G.get_edge_data(origin, dest)['weight']
        nx.set_edge_attributes(G, {(origin, dest):{'weight': (prev_weight + 1)}})
    else:
        G.add_edge(origin, dest, weight=1)
                                   
    return G
    
def node_time_step(G, origin, a, b, c, d, e):
    net_payoffs = get_net_payoff(G, origin, a, b, c, d)
    if decide_to_link(net_payoffs, e):
        dest = pick_dest_node(net_payoffs)
        return (origin, dest)
    
    else:
        return None
    
    

def time_step(G, a, b, c, d, e):
    edges_to_add = []
    for node in G.nodes:
        edge = node_time_step(G, node, a, b, c, d, e)
        if edge is not None:
            edges_to_add.append(edge)
            
    for edge in edges_to_add:
        G = add_edge(G, edge[0], edge[1])
        
    return G
    

In [199]:
# parameter values
a, b, c, d, e = 1, 1, 0.75, 1, 0.5

# number of periods to run simulation
n_steps = 10

# number of simulation nodes
n_high = 5
n_low = 5

# model gdp values
gdp_high = 10
gdp_low = 1

def run_simulation(n_steps, n_high, n_low, gdp_high, gdp_low, a=a, b=b, c=c, d=d, e=e):
    G = init_model_network(n_high, n_low, gdp_high, gdp_low)
    for step in range(n_steps):
        G = time_step(G, a, b, c, d, e)
        
    return G

In [200]:
G = run_simulation(n_steps, n_high, n_low, gdp_high, gdp_low)

In [201]:
G.edges(data=True)

OutEdgeDataView([(0, 2, {'weight': 2}), (1, 2, {'weight': 3}), (2, 0, {'weight': 4}), (2, 1, {'weight': 1}), (3, 2, {'weight': 4}), (3, 0, {'weight': 1}), (4, 2, {'weight': 2}), (5, 2, {'weight': 3}), (5, 0, {'weight': 2}), (6, 1, {'weight': 2}), (6, 2, {'weight': 4}), (7, 1, {'weight': 1}), (7, 0, {'weight': 2}), (7, 2, {'weight': 3}), (8, 2, {'weight': 4}), (9, 2, {'weight': 4})])

In [202]:
list(nx.strongly_connected_components(G))

[{0, 1, 2}, {3}, {4}, {5}, {6}, {7}, {8}, {9}]

In [184]:
list(nx.weakly_connected_components(G))

[{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}]